# **Automation of the CLI commands of the Snakemake workflow**

This Jupyter Notebook has the objective of automating Snakemake commands.

While it can be useful for quick tasks, using this instead of the command line may not provide you with the ability to access some of Snakemake's advanced features.

* Environment to use: [Greference_tools](code/environments/Greference_tools.yml)

In [ ]:
## Libreries
import subprocess
import os
import re

In [ ]:
## lists for the loops
chr_list        = ["chr3", "chr5", "chr7", "chr12", "chr17"]

sample_list     = ["ERR696683", "ERR753368", "ERR753369" ,"ERR753370" ,"ERR753371" ,"ERR753372", 
                  "ERR753373", "ERR753374", "ERR753375", "ERR753376", "ERR753377", "ERR753378"]


1. **rule download_data**

In [ ]:
subprocess.run(["snakemake", "--cores", "1", "download_data"])

* rule pre_processing

Filtering the chromosomes, make sure that the chromosomes of the config file are **ch3**. 

In [ ]:
## Running the 3º chromosome
subprocess.run(["snakemake", "--cores", "1", "data/original_bam/filtering/ERR696683_chr3_sorted.bam"])

## Running the rest of the chromosomes
for chr in range(len(chr_list) - 1):
    chr1 = chr_list[chr]
    chr2 = chr_list[chr + 1]
    
    print("""
          Chromosomes transformed:
          """, chr1, chr2)
    
    subprocess.run(["sed", "-i", "6,+12s/" + chr1 + "/" + chr2 + "/g", "config.yaml"])
    subprocess.run(["sed", "-i", "39s/" + chr1 + "/" + chr2 + "/g", "config.yaml"])

    subprocess.run(["snakemake", "--cores", "1", "data/original_bam/filtering/ERR696683_" + chr2 + "_sorted.bam"])

In [ ]:
## reset config 
subprocess.run(["sed", "-i", "6,+12s/chr17/chr3/g", "config.yaml"])
subprocess.run(["sed", "-i", "39s/chr17/chr3/g", "config.yaml"])

## Function for the next rules:

* reference_genome 
* fastqc 
* fastp
* fastqc_trimmed
* bwa_mapping 
* sam_to_bam 
* delete_duplicates 
* extracting_variants 
* vep_install 
* vep_cli 
* parsing_dataR

In [ ]:
def snake_workflow(chr: str):
    ## Running the workflow for x chormosome                                                                       
    if chr == "chr3":
        subprocess.run(["snakemake", "--cores", "1", "data/reference/genome.fa"])
    elif chr == "chr5" or chr == "chr7" or chr == "chr12" or chr == "chr17":
        subprocess.run(["snakemake", "--cores", "1", "--force", "data/reference/genome.fa"]) 
    
    for sample in sample_list:
        ## Quality inspection
        subprocess.run([
            "snakemake", "--cores", "1", "--use-conda",
            "results/fastqc_result/" + sample + "_" + chr + "_1_fastqc.html",   
            "data/processed/" + sample + "_" + chr + "_1_fastp.fastq.gz",   
            "results/fastqc_result/trimmed/" + sample + "_" + chr + "_1_fastp_fastqc.html"
        ])
        
    for sample in sample_list:
        ## Mapping reads
        subprocess.run([
            "snakemake", "--cores", "1", "--use-conda",
            "results/mapped_reads/" + sample + "_" + chr + "_sorted.sam",   
            "results/mapped_reads/bam_files/" + sample + "_" + chr + "_sorted.bam",   
            "results/mapped_reads/bam_files/" + sample + "_" + chr + "_dedup.bam"
        ])
        
    for sample in sample_list:
        # Extracting variants  
        subprocess.run([
            "snakemake", "--cores", "1", "--use-conda", 
            "results/variants/" + sample + "_" + chr + ".vcf"
        ])   
    
    ## Downloading VEP 
    if chr == "chr3": 
        subprocess.run(["snakemake", "--cores", "1", "--use-conda", "vep_install_db"])

    ## VEP CLI
    for sample in sample_list:
        subprocess.run([
            "snakemake", "--cores", "1", "--use-conda", 
            "results/variants/vep/" + sample + "_" + chr + ".txt"
        ])
    
    ## Extracting the gene from the VEP files
    subprocess.run(["snakemake", "--cores", "1", "parsing_dataR"])


## Chromosome 3

In [ ]:
snake_workflow("chr3")

## Changing the config file to do the chromosome 5

In [ ]:
changes_chr3_to_5 = ["6,+12s/chr3/chr5/g", "24,+12s/chr3/chr5/g", "39s/chr3/chr5/g",
                     "46s/chromosome.3.fa.gz/chromosome.5.fa.gz/g", "65s/PIK3CA/APC/g"]
for change in changes_chr3_to_5:
    subprocess.run(["sed", "-i", change, "config.yaml"])

In [ ]:
snake_workflow("chr5")

## Changing the config file to do the chromosome 7 

In [ ]:
changes_chr5_to_7 = ["6,+12s/chr5/chr7/g", "24,+12s/chr5/chr7/g", "39s/chr5/chr7/g",
                     "46s/chromosome.5.fa.gz/chromosome.7.fa.gz/g", "65s/APC/BRAF/g"]
for change in changes_chr5_to_7:
    subprocess.run(["sed", "-i", change, "config.yaml"])   

In [ ]:
snake_workflow("chr7")

## Changing the config file to do the chromosome 12 

In [ ]:
changes_chr7_to_12 = ["6,+12s/chr7/chr12/g", "24,+12s/chr7/chr12/g", "39s/chr7/chr12/g",
                      "46s/chromosome.7.fa.gz/chromosome.12.fa.gz/g", "65s/BRAF/KRAS/g"]
for change in changes_chr7_to_12:
    subprocess.run(["sed", "-i", change, "config.yaml"])     

In [ ]:
snake_workflow("chr12")

## Changing the config file to do the chromosome 17 

In [ ]:
changes_chr12_to_17 = ["6,+12s/chr12/chr17/g", "24,+12s/chr12/chr17/g", "39s/chr12/chr17/g",
                       "46s/chromosome.12.fa.gz/chromosome.17.fa.gz/g", "65s/KRAS/TP53/g"]
for change in changes_chr12_to_17:
    subprocess.run(["sed", "-i", change, "config.yaml"])

In [ ]:
snake_workflow("chr17")

In [ ]:
## reset config 
changes_chr17_to_3 = ["6,+12s/chr17/chr3/g", "24,+12s/chr17/chr3/g", "39s/chr17/chr3/g",
                      "46s/chromosome.17.fa.gz/chromosome.3.fa.gz/g", "65s/TP53/PIK3CA/g"]
for change in changes_chr17_to_3:
    subprocess.run(["sed", "-i", change, "config.yaml"])

## **Last rule**

### **rule R_plotting**

Plotting the data of the 5 gene tables

In [ ]:
subprocess.run(["snakemake", "--cores", "1", "R_plotting"])